In [166]:
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data

In [185]:
happy = pd.read_csv('data/processed/df_tidy.csv').drop(['Unnamed: 0'],axis=1)
country_ids = pd.read_csv('data/raw/country-ids.csv')
 
country_ids = country_ids.append(
    pd.DataFrame([[48,  'Bahrain'],
                  [174, 'Comoros'],
                  [344, 'Hong Kong'],
                  [462, 'Maldives'],
                  [470, 'Malta'],
                  [480, 'Mauritius'],
                  [702, 'Singapore']
], columns = ["id", 'name']), ignore_index=True)

country_mapper = {
    r'^Bolivia.*'              : 'Bolivia',
    r'^Congo$'                 : 'Congo (Kinshasa)',
    r'^Congo.*Demo.*'          : 'Congo (Brazzaville)',
    r'^Cote d.*'               : 'Ivory Coast',
    r'.*Cyprus.*'              : 'Cyprus',
    r'^Hong Kong.*'            : 'Hong Kong',
    r'^Iran.*'                 : 'Iran',
    r'^Korea, R.*'             : 'South Korea',
    r'^Lao.*'                  : 'Laos',
    r'.*Macedonia.*'           : 'North Macedonia',
    r'^Moldova.*'              : 'Moldova',
    r'^Palestinian Territory.*': 'Palestinian Territories',
    r'^Russia.*'               : 'Russia',
    r'^Singapore.*'            : 'Singapore',
    r'^Syria.*'                : 'Syria',
    r'^Somali.*'               : 'Somalia',
    r'^Taiwan.*'               : 'Taiwan',
    r'^Tanzania.*'             : 'Tanzania',
    r'^Trinidad & Tobago'      : 'Trinidad and Tobago',
    r'^Venezuela.*'            : 'Venezuela',
    r'^Viet.*'                 : 'Vietnam'
}
happy = happy.replace(regex=country_mapper)
country_ids = country_ids.replace(regex=country_mapper)
happy = happy.merge(country_ids, left_on = "Country", right_on = 'name')
happy = happy.drop(columns = 'name')
happy['Delta_happy'] = happy['Happiness_score']
happy.head()


,Country,Happiness_rank,Happiness_score,GDP_per_capita,Social_support,Life_expectancy,Freedom,Generosity,Corruption,Year,id,Delta_happy
0,Switzerland,1,7.587,1.3965,1.3495,0.9414,0.6656,0.2968,0.4198,2015,756,7.587
1,Switzerland,2,7.509,1.5273,1.1452,0.8630,0.5856,0.2808,0.4120,2016,756,7.509
2,Switzerland,4,7.494,1.5650,1.5169,0.8581,0.6201,0.2905,0.3670,2017,756,7.494
3,Switzerland,5,7.487,1.4200,1.5490,0.9270,0.6600,0.2560,0.3570,2018,756,7.487
4,Switzerland,6,7.480,1.4520,1.5260,1.0520,0.5720,0.2630,0.3430,2019,756,7.480


In [56]:
world_map = alt.topo_feature(data.world_110m.url, 'countries')
world_map

UrlData({
  format: TopoDataFormat({
    feature: 'countries',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'
})

In [187]:
map_click = alt.selection_multi(fields=['id'])

alt.Chart(world_map).mark_geoshape(
    stroke='black', 
    strokeWidth=0.5
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data = happy, 
                         key ='id', 
                         fields = ['Country',
                                   'Delta_happy',
                                   'Happiness_rank'])
).encode(alt.Color('Delta_happy:Q', 
                   scale=alt.Scale(domain=[0, 10], 
                                   scheme = "redyellowgreen"),
                   legend=alt.Legend(title = "Happiness")),
         opacity=alt.condition(map_click, alt.value(1), alt.value(0.2)),
         tooltip=[alt.Tooltip(field = 'Country',
                              type = 'nominal',
                              title = 'Country'), 
                  alt.Tooltip(field = 'Delta_happy',
                              type = "quantitative",
                              title = 'Happiness'),
                  alt.Tooltip(field = 'Happiness_rank',
                              type = "quantitative",
                              title = 'Rank')]
).add_selection(map_click
).project(type='naturalEarth1'
).properties(
    width=700,
    height=350
)

alt.Chart(...)

In [189]:
happy.to_csv("data/processed/extra_clean.csv")

In [163]:
c2 = country_ids.copy()